In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Data processing

In [ ]:
data = pd.read_csv("mnist_train.csv",header=None)

In [ ]:
x_train = data.iloc[:,1:]
y_train = data.iloc[:,0]

## Softmax Definition

In [ ]:
def softmax(X, W, b):
    logit = W @ X.t + b
    numerator = np.exp(logit)
    denominator =  np.sum(np.exp(logit),axis=1)
    ans = np.divide(numerator,denominator)
    return ans

## Loss Function Defintion

The loss function we select is the log cross entropy loss. It is defined as 
$$
L(x,y;W,b) = -ln(p_{y}) = - \sum_{i=0}^{K-1}ln(p_{i})
$$
where
$$
p_{y} = p(y = i|\textbf{x}) = \frac{e^{f_{i}}}{\sum_{i=0}^{K-1} e^{f_{k}}}
$$
and 
$$
f_{i} = w_{j}\textbf{x} + b_{b_{j}}
$$
To make the computation a bit easier, we can using a one hot encoding to mask all values not equal to the class in question to 0.

In [ ]:
def one_hot(n,K,Y):
    Y_onehot = np.zeros((n,K))
    for i in range(K):
        for j in range(n):
            if i != Y[j]:
                continue
            Y_onehot[j][i] = 1
    return Y_onehot

In [ ]:
def L(X, Y, W, b):
    K = W.shape[0]
    n = X.shape[0]
    Y_onehot = one_hot(n,K,Y)
    P = softmax(X,W,b)

    L = np.sum((Y_onehot * np.log(P)) * -1)

    return L

## Loss Function Gradient
The gradient is given by 
$$
\frac{\partial L}{\partial W} = (P - Y)^TX
$$
and 
$$
\frac{\partial L}{\partial \textbf{b}} = (P - Y)^T \textbf{b}
$$

In [ ]:
def L_prime(X, Y, W, b):
    K = W.shape
    n = X.shape[0]
    P = softmax(X,W,b)
    Y_onehot = one_hot(n,K,Y)
    dL_by_dW =  ((P - Y_onehot).T).dot(X)
    one = np.ones((n,1))
    dL_by_db =  ((P - Y_onehot).T).dot(one)
    return dL_by_dW, dL_by_db

## Network Defintion

## Training Loop

In [ ]:
K, D = x_train.shape
learning_rate = 0.0025
epochs = 1      # Number of iterations
W = np.zeros((K,D))   # Weight matrix.
b = np.zeros((K,1))   # Bias vector.

# We will keep track of training loss over iterations.
iterations = [0]
L_list = [L(x_train, y_train, W, b)]
# for epoch in range(epochs):
for i in range(len(x_train)):
    gradient_W, gradient_b = L_prime(x_train[i], y_train[i],W,b)
    W_new = W - learning_rate * gradient_W
    b_new = b - learning_rate * gradient_b

    iterations.append(i+1)
    print("hello")
    L_list.append(L(x_train[i], y_train[i], W_new, b_new))
    norm = np.abs(W_new-W).sum() + np.abs(b_new-b).sum()  # L1-norm as jumping out criteria.
    if norm < 0.0005:
        print("Gradient descent has converged after " + str(i) + " iterations.")
        break
    W = W_new
    b = b_new
    
print ('W matrix: \n' + str(W))
print ('b vector: \n' + str(b))

## Training Loss Curve

In [ ]:
plt.title('Training curve')
plt.xlabel('iteration')
plt.ylabel('L(W,b)')
plt.semilogy(iterations, np.array(L_list).reshape(-1, 1))
plt.show()